In [0]:
from numpy import genfromtxt
import pandas as pd

In [0]:
from google.colab import files
uploaded = files.upload()
import math

KeyboardInterrupt: ignored

In [0]:
#import tensorflow as tf
from indrnn_cell import indnn_cell
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np

from ind_rnn_cell import IndRNNCell

# Parameters taken from https://arxiv.org/abs/1803.04831
TIME_STEPS = 2
NUM_UNITS = 128
LEARNING_RATE_INIT = 0.0002
LEARNING_RATE_DECAY_STEPS = 20000
RECURRENT_MAX = pow(2, 1 / TIME_STEPS)

# Parameters taken from https://arxiv.org/abs/1511.06464
BATCH_SIZE = 28

Instructions for updating:
non-resource variables are not supported in the long term


In [0]:
data = genfromtxt('Route251.csv', delimiter=',')
data = data[1:]
dat = pd.read_csv('Route251.csv')
arr = dat.Timeslot.unique()
arr

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23,  0,  1])

In [0]:
def get_batch(j):
  """Generate the adding problem dataset"""
  # # Build the first sequence
  # add_values = np.random.rand(BATCH_SIZE, TIME_STEPS)
  # # Build the second sequence with one 1 in each half and 0s otherwise
  # add_indices = np.zeros_like(add_values)
  # half = int(TIME_STEPS / 2)
  # for i in range(BATCH_SIZE):
  #   first_half = np.random.randint(half)
  #   second_half = np.random.randint(half, TIME_STEPS)
  #   add_indices[i, [first_half, second_half]] = 1

  # # Zip the values and indices in a third dimension:
  # # inputs has the shape (batch_size, time_steps, 2)
  # inputs = np.dstack((add_values, add_indices))
  # targets = np.sum(np.multiply(add_values, add_indices), axis=1)
  c  = data[:,:][data[:, 0] == j]
  targets = []
  inputs = []
  for i in c :
    d= []
    d.append(i[0])
    d.append(i[2])
    inputs.append(d)
    targets.append(i[1])
  inputs =  np.array(inputs)
  inputs = np.dstack(inputs).T
  targets = np.array(targets) 
  return inputs, targets

def predict(los) :
  trips = []
  for i in range(0,len(los)) :
    trips.append(indnn_cell[i])
  return trips  

In [0]:
inputs_ph = tf.placeholder(tf.float32, shape=(BATCH_SIZE, TIME_STEPS, 1))
targets_ph = tf.placeholder(tf.float32, shape=BATCH_SIZE)

# Build the graph
first_input_init = tf.random_uniform_initializer(-RECURRENT_MAX,
                                                  RECURRENT_MAX)
first_layer = IndRNNCell(NUM_UNITS, recurrent_max_abs=RECURRENT_MAX,
                          recurrent_kernel_initializer=first_input_init)
second_layer = IndRNNCell(NUM_UNITS, recurrent_max_abs=RECURRENT_MAX)

cell = tf.nn.rnn_cell.MultiRNNCell([first_layer, second_layer])
# cell = tf.nn.rnn_cell.BasicLSTMCell(NUM_UNITS) uncomment this for LSTM runs

output, state = tf.nn.dynamic_rnn(cell, inputs_ph, dtype=tf.float32)
last = output[:, -1, :]

weight = tf.get_variable("softmax_weight", shape=[NUM_UNITS, 1])
bias = tf.get_variable("softmax_bias", shape=[1],
                        initializer=tf.constant_initializer(0.1))
prediction = tf.squeeze(tf.matmul(last, weight) + bias)

loss_op = tf.losses.mean_squared_error(tf.squeeze(targets_ph), prediction)

global_step = tf.get_variable("global_step", shape=[], trainable=False,
                              initializer=tf.zeros_initializer)
learning_rate = tf.train.exponential_decay(LEARNING_RATE_INIT, global_step,
                                            LEARNING_RATE_DECAY_STEPS, 0.1,
                                            staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate)
optimize = optimizer.minimize(loss_op, global_step=global_step)




Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [0]:
# Train the model
los = []
for j in arr :
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    step = 0
    losses = []
    for i in range(10):
      for _ in range(100):
        # Generate new input data
        inputs, targets = get_batch(j)
        loss, _ = sess.run([loss_op, optimize],
                            {inputs_ph: inputs, targets_ph: targets})
        losses.append(loss)
        step += 1
        print("Step [x100] {} MSE {}".format(int(step / 100), np.mean(losses)))
  los.append(losses[-1])
  trips = predict(los)

Streaming output truncated to the last 5000 lines.
Step [x100] 0 MSE 19.461572647094727
Step [x100] 0 MSE 19.37386131286621
Step [x100] 0 MSE 19.28053855895996
Step [x100] 0 MSE 19.17653465270996
Step [x100] 0 MSE 19.05963706970215
Step [x100] 0 MSE 18.928617477416992
Step [x100] 0 MSE 18.782833099365234
Step [x100] 0 MSE 18.621427536010742
Step [x100] 0 MSE 18.444141387939453
Step [x100] 0 MSE 18.25080108642578
Step [x100] 0 MSE 18.041362762451172
Step [x100] 0 MSE 17.815889358520508
Step [x100] 0 MSE 17.574556350708008
Step [x100] 0 MSE 17.317665100097656
Step [x100] 0 MSE 17.045612335205078
Step [x100] 0 MSE 16.75895118713379
Step [x100] 0 MSE 16.458364486694336
Step [x100] 0 MSE 16.144474029541016
Step [x100] 0 MSE 15.818227767944336
Step [x100] 0 MSE 15.480779647827148
Step [x100] 0 MSE 15.133469581604004
Step [x100] 0 MSE 14.77783489227295
Step [x100] 0 MSE 14.415596008300781
Step [x100] 0 MSE 14.048664093017578
Step [x100] 0 MSE 13.679123878479004
Step [x100] 0 MSE 13.3092174530

In [0]:
trips

[1.0,
 3.0,
 4.0,
 4.0,
 5.0,
 6.0,
 5.0,
 6.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 4.0,
 4.0,
 3.0,
 4.0,
 1.0,
 1.0]